In [121]:
import pandas as pd
import streamlit_authenticator as stauth
import sqlite3
import string
import random
import hashlib


conn_sqlite = sqlite3.connect("credentials.db")
c = conn_sqlite.cursor()

In [122]:
def make_hashes(password):
	return hashlib.sha256(str.encode(password)).hexdigest()

In [123]:
def generate_random_string(length):
    random.seed(10)
    letters = string.ascii_lowercase + string.ascii_uppercase + string.digits
    return ''.join(random.choice(letters) for _ in range(length))

In [124]:
def makehash(x):
    passw = stauth.Hasher(x).generate()
    return passw

In [125]:
makehash(['sugengw07'])

['$2b$12$7cJULG9oBIBmmACraoLgNudBODfxNbXLK./osgWIVgk2d/6mAQ6hm']

In [126]:
makehash(['sugengw07'])

['$2b$12$KrCrxwowUUnx33cdjD.vVOdKE5SpKJC3HIagVpRjVFEhBeLbRJIwe']

In [127]:
data = pd.read_excel('pegawai110.xlsx', dtype={'NIP':'str','KD':'str'},usecols=['NIP','Nama','Kantor','KD'])

In [128]:
data['password'] = data['NIP'].apply(lambda x: x +'@'+ generate_random_string(4))

In [129]:
data['password_hashed'] = data['password'].apply(lambda x: make_hashes(x))

In [130]:
data.to_excel('login.xlsx',index=False)

In [131]:
data.rename(columns={'Nama':'names', 'NIP':'usernames', 'Kantor':'kantor', 'KD':'kd'},inplace=True)

In [132]:
data= data[['usernames','names','password_hashed','kd','kantor']]

In [133]:
c.execute('CREATE TABLE IF NOT EXISTS user(usernames text, names text,password text, kd text, kantor text)')

In [134]:
def add_user(nip, nama, password,kode,kpp):
    c.execute('INSERT INTO user(usernames, names, password,kd,kantor) VALUES(?,?,?,?,?)',(nip,nama, password,kode,kpp))
    conn_sqlite.commit()

In [135]:
usernames = data['usernames'].tolist()
names = data['names'].tolist()
password = data['password_hashed'].tolist()
kd = data['kd'].tolist()
kantor = data['kantor'].tolist()
for (username,name,passw,kode,kpp) in zip(usernames,names,password, kd, kantor):
    add_user(username,name,passw,kode,kpp)
    

In [111]:
# c.execute('DROP TABLE user')

In [ ]:
nip = [x['NIP'] for x in data]
nama = [x['Nama'] for x in data]
kantor = [x['Kantor'] for x in data]
password = [x['password_hashed'] for x in data]

In [ ]:
add_user()

In [ ]:
data['password_hashed']='x '
for idx,x in enumerate(data['password']):
    print(data.loc[idx,'password_hashed':'password_hashed'])

In [ ]:
data = data.head()

In [ ]:
conn_sqlite.close()

In [ ]:
data['pass_hashed'] = data['password'].apply(lambda x: makehash([x])[0])

In [ ]:
secrets = pd.DataFrame(pass_hashed, index=data.NIP.tolist(),columns=['PASS_HASHED'])

In [ ]:
credential = pd.merge(data, secrets, left_on='NIP', right_index=True, how='inner')

In [ ]:
credential =data

In [ ]:
import duckdb
db = duckdb.connect('credentials.duckdb')

In [ ]:
db.execute('drop table credential ')

In [ ]:
db.execute('create table credentials from credential')

In [ ]:
db.close()

In [ ]:
s=db.execute('select * from credential').df()


In [ ]:
for passq in s['pass_hashed']:
    print(passq)